# ResearchRAG - Modulares RAG-System

Dieses Notebook steuert das modulare RAG-System für die wissenschaftliche Studie.

## 🎯 Überblick

- **Zweck**: Vergleich verschiedener RAG-Komponenten
- **Team**: 4 Personen, 30 Tage
- **Daten**: DSGVO-Text
- **Ziel**: Systematische Evaluation verschiedener Ansätze

## 📋 Notebook-Struktur

1. **Setup & Installation** - Abhängigkeiten und Umgebung
2. **Konfiguration** - Pipeline-Konfigurationen auswählen
3. **Datenloading** - DSGVO-Dokumente laden
4. **Pipeline-Erstellung** - RAG-Pipeline initialisieren
5. **Indexierung** - Dokumente verarbeiten und indexieren
6. **Querying** - Interaktive Abfragen
7. **Evaluation** - Systematische Bewertung
8. **Analyse** - Komponenten-Vergleich und Statistiken

# 1. Setup & Installation

## Google Colab Setup

Falls Sie in Google Colab arbeiten, führen Sie zuerst diese Zellen aus:

In [ ]:
# Google Colab Setup - nur ausführen wenn in Colab
import sys
import os

# Prüfen ob wir in Google Colab sind
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Google Colab erkannt - Setup wird gestartet...")
    
    # Google Drive mounten für Persistierung
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Arbeitsverzeichnis erstellen
    project_dir = '/content/drive/MyDrive/FOM_RAG_Project'
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)
    os.chdir(project_dir)
    
    # Projekt-Repository klonen (falls noch nicht vorhanden)
    if not os.path.exists('src'):
        print("📥 Lade Projekt-Code...")
        # Hier würden Sie normalerweise das Repository klonen
        # !git clone https://github.com/your-repo/FOM.BigDataAnalyseProjekt.git .
        print("⚠️  Bitte laden Sie die Projekt-Dateien manuell hoch")
    
    print("✅ Google Colab Setup abgeschlossen")
    print(f"📁 Arbeitsverzeichnis: {os.getcwd()}")
else:
    print("💻 Lokale Umgebung erkannt")
    print(f"📁 Arbeitsverzeichnis: {os.getcwd()}")


In [ ]:
# Package Installation
import subprocess
import sys

def install_package(package):
    """Installiert ein Package falls nicht vorhanden."""
    try:
        __import__(package.split('>=')[0].split('==')[0])
        print(f"✅ {package} bereits installiert")
    except ImportError:
        print(f"📦 Installiere {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} installiert")

# Kern-Abhängigkeiten installieren
core_packages = [
    "numpy>=1.24.0",
    "pandas>=2.0.0", 
    "scikit-learn>=1.3.0",
    "openai>=1.0.0",
    "tqdm>=4.65.0",
    "python-dotenv>=1.0.0",
    "matplotlib>=3.7.0",
    "seaborn>=0.12.0"
]

print("🔧 Installiere Kern-Abhängigkeiten...")
for package in core_packages:
    install_package(package)

print("✅ Alle Abhängigkeiten installiert!")


In [ ]:
# Umgebungsvariablen und API-Schlüssel
import os
from dotenv import load_dotenv
import getpass

# .env Datei laden (falls vorhanden)
load_dotenv()

# OpenAI API Key setup
if not os.getenv("OPENAI_API_KEY"):
    print("🔑 OpenAI API Key erforderlich")
    print("Sie können den Key auf verschiedene Weise setzen:")
    print("1. Über .env Datei: OPENAI_API_KEY=your_key_here")
    print("2. Über Umgebungsvariable: export OPENAI_API_KEY=your_key_here")
    print("3. Hier direkt eingeben (nur für Tests!):")
    
    api_key = getpass.getpass("OpenAI API Key eingeben (wird versteckt): ")
    if api_key:
        os.environ["OPENAI_API_KEY"] = api_key
        print("✅ API Key gesetzt")
    else:
        print("⚠️  Kein API Key eingegeben - OpenAI-Komponenten werden nicht funktionieren")
else:
    print("✅ OpenAI API Key gefunden")

# Andere Umgebungsvariablen
print("\n📋 Aktuelle Umgebung:")
print(f"  Python Version: {sys.version}")
print(f"  Arbeitsverzeichnis: {os.getcwd()}")
print(f"  OpenAI API Key: {'✅ Gesetzt' if os.getenv('OPENAI_API_KEY') else '❌ Nicht gesetzt'}")


In [ ]:
# 2. Konfiguration

## Pipeline-Konfigurationen auswählen

Hier können Sie verschiedene Konfigurationen für Ihr Experiment auswählen.


In [ ]:
# Imports für das RAG-System
import sys
import os

# Pfad zum src-Verzeichnis hinzufügen
if 'src' not in sys.path:
    sys.path.append('src')

# RAG-System Imports
from config.pipeline_configs import get_baseline_config, get_alternative_configs
from config.experiment_configs import get_experiment_configs
from core.rag_pipeline import RAGPipeline
from utils.data_loader import DataLoader

print("✅ RAG-System Module importiert")

# Verfügbare Konfigurationen anzeigen
print("\n📋 Verfügbare Pipeline-Konfigurationen:")

# Baseline-Konfiguration
baseline_config = get_baseline_config()
print(f"\n🔹 Baseline: {baseline_config.get_component_types()}")

# Alternative Konfigurationen
alternative_configs = get_alternative_configs()
for name, config in alternative_configs.items():
    print(f"🔹 {name}: {config.get_component_types()}")

print(f"\n📊 Insgesamt {1 + len(alternative_configs)} Konfigurationen verfügbar")


In [ ]:
# Konfiguration auswählen
import ipywidgets as widgets
from IPython.display import display, clear_output

# Dropdown für Konfigurationsauswahl
config_names = ["baseline"] + list(alternative_configs.keys())
config_dropdown = widgets.Dropdown(
    options=config_names,
    value="baseline",
    description="Konfiguration:",
    style={'description_width': 'initial'}
)

# Aktuelle Konfiguration anzeigen
config_output = widgets.Output()

def on_config_change(change):
    with config_output:
        clear_output()
        config_name = change['new']
        
        if config_name == "baseline":
            selected_config = baseline_config
        else:
            selected_config = alternative_configs[config_name]
        
        print(f"📋 Gewählte Konfiguration: {config_name}")
        print(f"🔧 Komponenten: {selected_config.get_component_types()}")
        
        # Detaillierte Konfiguration anzeigen
        print("\n📄 Detaillierte Konfiguration:")
        for component_type in ["chunker", "embedding", "vector_store", "language_model"]:
            config_method = getattr(selected_config, f"get_{component_type}_config")
            component_config = config_method()
            print(f"  {component_type}: {component_config}")

config_dropdown.observe(on_config_change, names='value')

# Initial anzeigen
on_config_change({'new': config_dropdown.value})

display(config_dropdown, config_output)

# Aktuelle Konfiguration für spätere Verwendung speichern
def get_current_config():
    config_name = config_dropdown.value
    if config_name == "baseline":
        return baseline_config
    else:
        return alternative_configs[config_name]

print("\n✅ Konfiguration bereit - verwenden Sie get_current_config() um die aktuelle Konfiguration zu erhalten")


In [ ]:
# 3. Datenloading

## DSGVO-Dokumente laden

Hier laden wir die DSGVO-Dokumente, die als Basis für unser RAG-System dienen.


In [ ]:
# Datenverzeichnis prüfen und erstellen
import os
from pathlib import Path

# Datenverzeichnis-Struktur erstellen
data_dirs = [
    "data/raw",
    "data/processed/chunks", 
    "data/processed/embeddings",
    "data/evaluation/results"
]

for dir_path in data_dirs:
    Path(dir_path).mkdir(parents=True, exist_ok=True)

print("✅ Datenverzeichnisse erstellt")

# DSGVO-Datei prüfen
dsgvo_file = "data/raw/dsgvo.txt"

if os.path.exists(dsgvo_file):
    with open(dsgvo_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    print(f"📄 DSGVO-Datei gefunden: {dsgvo_file}")
    print(f"📊 Dateigröße: {len(content):,} Zeichen")
    print(f"📝 Erste 200 Zeichen:")
    print(content[:200] + "...")
    
else:
    print(f"⚠️  DSGVO-Datei nicht gefunden: {dsgvo_file}")
    print("💡 Bitte laden Sie die DSGVO-Datei in das data/raw/ Verzeichnis")
    print("📥 Sie können sie hier herunterladen: https://eur-lex.europa.eu/legal-content/DE/TXT/?uri=CELEX%3A32016R0679")
    
    # Beispiel-Inhalt für Testzwecke
    sample_content = """
    VERORDNUNG (EU) 2016/679 DES EUROPÄISCHEN PARLAMENTS UND DES RATES
    
    Artikel 1 - Gegenstand und Ziele
    
    (1) Diese Verordnung enthält Vorschriften zum Schutz natürlicher Personen bei der Verarbeitung personenbezogener Daten und zum freien Verkehr solcher Daten.
    
    (2) Diese Verordnung schützt die Grundrechte und Grundfreiheiten natürlicher Personen und insbesondere deren Recht auf Schutz personenbezogener Daten.
    
    Artikel 83 - Allgemeine Bedingungen für die Verhängung von Geldbußen
    
    (1) Jede Aufsichtsbehörde stellt sicher, dass die Verhängung von Geldbußen gemäß diesem Artikel für Verstöße gegen diese Verordnung gemäß den Absätzen 4, 5 und 6 in jedem Einzelfall wirksam, verhältnismäßig und abschreckend ist.
    
    (5) Verstöße gegen die folgenden Bestimmungen werden im Einklang mit Absatz 2 mit Geldbußen von bis zu 20 000 000 EUR oder im Fall eines Unternehmens von bis zu 4 % seines gesamten weltweit erzielten Jahresumsatzes des vorangegangenen Geschäftsjahrs verhängt, je nachdem, welcher Betrag höher ist:
    """
    
    # Beispiel-Datei erstellen
    with open(dsgvo_file, 'w', encoding='utf-8') as f:
        f.write(sample_content)
    
    print(f"📝 Beispiel-DSGVO-Datei erstellt: {dsgvo_file}")
    print("⚠️  Dies ist nur ein Beispiel - bitte ersetzen Sie durch die vollständige DSGVO")


In [ ]:
# 4. Pipeline-Erstellung

## RAG-Pipeline initialisieren

Hier erstellen wir die RAG-Pipeline mit der gewählten Konfiguration.


In [ ]:
# RAG-Pipeline erstellen
import time

print("🔧 Erstelle RAG-Pipeline...")

# Aktuelle Konfiguration laden
current_config = get_current_config()
print(f"📋 Verwende Konfiguration: {current_config.get_component_types()}")

# Pipeline initialisieren
try:
    start_time = time.time()
    pipeline = RAGPipeline(current_config)
    init_time = time.time() - start_time
    
    print(f"✅ Pipeline erfolgreich erstellt in {init_time:.2f}s")
    
    # Pipeline-Informationen anzeigen
    pipeline_info = pipeline.get_pipeline_info()
    print(f"\n📊 Pipeline-Informationen:")
    print(f"  Status: {'🟢 Bereit' if pipeline else '🔴 Fehler'}")
    print(f"  Indexiert: {'✅ Ja' if pipeline.is_indexed else '❌ Nein'}")
    print(f"  Dokumente: {pipeline.indexed_document_count}")
    
    # Komponenten-Informationen
    component_info = pipeline.get_component_info()
    print(f"\n🔧 Komponenten-Details:")
    for component, info in component_info.items():
        print(f"  {component}: {info}")
        
except Exception as e:
    print(f"❌ Fehler beim Erstellen der Pipeline: {e}")
    print("💡 Prüfen Sie:")
    print("  - OpenAI API Key ist gesetzt")
    print("  - Alle Abhängigkeiten sind installiert")
    print("  - Konfiguration ist korrekt")
    
    # Traceback für Debugging
    import traceback
    print(f"\n🐛 Detaillierter Fehler:")
    traceback.print_exc()


In [ ]:
# 5. Indexierung

## Dokumente verarbeiten und indexieren

Hier werden die DSGVO-Dokumente in die Pipeline geladen und indexiert.


In [ ]:
# Dokumente laden und indexieren
import time

if 'pipeline' not in locals():
    print("❌ Pipeline nicht verfügbar - bitte führen Sie zuerst die Pipeline-Erstellung aus")
else:
    print("📥 Lade DSGVO-Dokumente...")
    
    try:
        # Dokumente aus Datei laden
        documents = pipeline.load_documents_from_file(dsgvo_file)
        print(f"✅ {len(documents)} Dokument(e) geladen")
        
        # Indexierung starten
        print("\n🔄 Starte Indexierung...")
        print("  Dies kann einige Minuten dauern, abhängig von der Dokumentgröße und den gewählten Komponenten")
        
        start_time = time.time()
        indexing_stats = pipeline.index_documents(documents, show_progress=True)
        total_time = time.time() - start_time
        
        print(f"\n✅ Indexierung abgeschlossen!")
        print(f"⏱️  Gesamtzeit: {total_time:.2f}s")
        
        # Detaillierte Statistiken
        print(f"\n📊 Indexierungs-Statistiken:")
        print(f"  📄 Dokumente: {indexing_stats['total_documents']}")
        print(f"  📝 Chunks: {indexing_stats['total_chunks']}")
        print(f"  🔢 Embeddings: {indexing_stats['total_embeddings']}")
        print(f"  📏 Embedding-Dimension: {indexing_stats['embedding_dimension']}")
        print(f"  ⚡ Chunks/Sekunde: {indexing_stats['chunks_per_second']:.1f}")
        print(f"  📐 Ø Chunk-Länge: {indexing_stats['average_chunk_length']:.0f} Zeichen")
        
        # Speichernutzung (geschätzt)
        embedding_size_mb = (indexing_stats['total_embeddings'] * indexing_stats['embedding_dimension'] * 4) / (1024 * 1024)
        print(f"  💾 Geschätzte Embedding-Größe: {embedding_size_mb:.1f} MB")
        
    except Exception as e:
        print(f"❌ Fehler bei der Indexierung: {e}")
        print("💡 Mögliche Ursachen:")
        print("  - Datei nicht gefunden oder nicht lesbar")
        print("  - OpenAI API Fehler (Rate Limit, Authentifizierung)")
        print("  - Speicher-/Netzwerkprobleme")
        
        import traceback
        print(f"\n🐛 Detaillierter Fehler:")
        traceback.print_exc()


In [ ]:
# 6. Querying

## Interaktive Abfragen

Jetzt können Sie Fragen zur DSGVO stellen und die Antworten des RAG-Systems testen.


In [ ]:
# Beispiel-Abfragen
example_questions = [
    "Was ist die maximale Geldbuße nach Art. 83 DSGVO?",
    "Welche Rechte haben betroffene Personen?",
    "Was ist eine Datenschutz-Folgenabschätzung?",
    "Wann ist eine Einwilligung erforderlich?",
    "Was sind die Grundsätze der Datenverarbeitung?"
]

print("🔍 Beispiel-Fragen zur DSGVO:")
for i, question in enumerate(example_questions, 1):
    print(f"  {i}. {question}")

print("\n💡 Sie können diese Fragen verwenden oder eigene stellen")

# Funktion für einzelne Abfragen
def ask_question(question, top_k=5, return_context=False):
    """Stellt eine Frage an die RAG-Pipeline."""
    if 'pipeline' not in locals() and 'pipeline' not in globals():
        print("❌ Pipeline nicht verfügbar")
        return None
    
    if not pipeline.is_indexed:
        print("❌ Pipeline ist nicht indexiert")
        return None
    
    print(f"❓ Frage: {question}")
    print("🔄 Verarbeite...")
    
    try:
        start_time = time.time()
        result = pipeline.query(question, top_k=top_k, return_context=return_context)
        query_time = time.time() - start_time
        
        print(f"\n✅ Antwort (in {query_time:.2f}s):")
        print(f"📝 {result['answer']}")
        
        if return_context:
            print(f"\n📚 Verwendete Quellen ({len(result['context'])}):")
            for i, context in enumerate(result['context'], 1):
                print(f"  {i}. {context[:100]}...")
        
        print(f"\n📊 Metadaten:")
        print(f"  ⏱️  Query-Zeit: {query_time:.2f}s")
        print(f"  🔍 Retrieval-Zeit: {result['retrieval_time']:.2f}s")
        print(f"  🤖 Generation-Zeit: {result['generation_time']:.2f}s")
        print(f"  📄 Gefundene Dokumente: {len(result['context'])}")
        
        return result
        
    except Exception as e:
        print(f"❌ Fehler bei der Abfrage: {e}")
        import traceback
        traceback.print_exc()
        return None

print("\n✅ Abfrage-Funktion bereit - verwenden Sie ask_question('Ihre Frage hier')")


In [ ]:
# Interaktive Abfrage-Widgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# Text-Input für Fragen
question_input = widgets.Text(
    value="Was ist die maximale Geldbuße nach Art. 83 DSGVO?",
    placeholder="Stellen Sie hier Ihre Frage zur DSGVO...",
    description="Frage:",
    layout=widgets.Layout(width='80%')
)

# Optionen
top_k_slider = widgets.IntSlider(
    value=5,
    min=1,
    max=20,
    step=1,
    description="Top-K:",
    tooltip="Anzahl der zu retrievenden Dokumente"
)

show_context_checkbox = widgets.Checkbox(
    value=False,
    description="Kontext anzeigen",
    tooltip="Zeigt die verwendeten Quellen an"
)

# Button für Abfrage
query_button = widgets.Button(
    description="Frage stellen",
    button_style='primary',
    icon='search'
)

# Output-Bereich
query_output = widgets.Output()

def on_query_button_click(b):
    """Behandelt Button-Klicks für Abfragen."""
    with query_output:
        clear_output()
        question = question_input.value.strip()
        
        if not question:
            print("❌ Bitte geben Sie eine Frage ein")
            return
        
        # Abfrage ausführen
        result = ask_question(
            question, 
            top_k=top_k_slider.value,
            return_context=show_context_checkbox.value
        )

query_button.on_click(on_query_button_click)

# Widget-Layout
query_widgets = widgets.VBox([
    widgets.HBox([question_input]),
    widgets.HBox([top_k_slider, show_context_checkbox]),
    widgets.HBox([query_button]),
    query_output
])

print("🔍 Interaktive Abfrage-Oberfläche:")
display(query_widgets)

# Tastenkombination für Enter
def on_question_submit(change):
    if change['type'] == 'change' and change['name'] == 'value':
        on_query_button_click(None)

# Enter-Taste aktivieren (funktioniert nur in einigen Jupyter-Umgebungen)
# question_input.observe(on_question_submit)


In [ ]:
# 7. Evaluation

## Systematische Bewertung

Hier führen wir eine systematische Evaluation mit dem QA-Datensatz durch.


In [ ]:
# QA-Datensatz laden
import json
import pandas as pd
from pathlib import Path

# QA-Datensatz-Pfad
qa_file = "data/evaluation/qa_pairs.json"

# Beispiel-QA-Datensatz erstellen falls nicht vorhanden
if not os.path.exists(qa_file):
    print("📝 Erstelle Beispiel-QA-Datensatz...")
    
    sample_qa_pairs = [
        {
            "id": "q1",
            "question": "Was ist die maximale Geldbuße nach Art. 83 DSGVO?",
            "expected_answer": "Die maximale Geldbuße beträgt 20 Millionen Euro oder 4% des weltweiten Jahresumsatzes",
            "category": "sanctions",
            "difficulty": "easy"
        },
        {
            "id": "q2", 
            "question": "Welche Rechte haben betroffene Personen nach der DSGVO?",
            "expected_answer": "Betroffene haben Rechte auf Auskunft, Berichtigung, Löschung, Einschränkung der Verarbeitung, Datenübertragbarkeit und Widerspruch",
            "category": "rights",
            "difficulty": "medium"
        },
        {
            "id": "q3",
            "question": "Was ist eine Datenschutz-Folgenabschätzung?",
            "expected_answer": "Eine Datenschutz-Folgenabschätzung ist eine Bewertung der Auswirkungen von Datenverarbeitungsvorgängen auf den Schutz personenbezogener Daten",
            "category": "compliance",
            "difficulty": "medium"
        },
        {
            "id": "q4",
            "question": "Wann ist eine Einwilligung zur Datenverarbeitung erforderlich?",
            "expected_answer": "Eine Einwilligung ist erforderlich, wenn keine andere Rechtsgrundlage nach Art. 6 DSGVO vorliegt",
            "category": "legal_basis",
            "difficulty": "hard"
        },
        {
            "id": "q5",
            "question": "Was sind die Grundsätze der Datenverarbeitung nach Art. 5 DSGVO?",
            "expected_answer": "Die Grundsätze umfassen Rechtmäßigkeit, Transparenz, Zweckbindung, Datenminimierung, Richtigkeit, Speicherbegrenzung und Integrität/Vertraulichkeit",
            "category": "principles",
            "difficulty": "hard"
        }
    ]
    
    # Datei erstellen
    Path(qa_file).parent.mkdir(parents=True, exist_ok=True)
    with open(qa_file, 'w', encoding='utf-8') as f:
        json.dump(sample_qa_pairs, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Beispiel-QA-Datensatz erstellt: {qa_file}")

# QA-Datensatz laden
with open(qa_file, 'r', encoding='utf-8') as f:
    qa_pairs = json.load(f)

print(f"📊 QA-Datensatz geladen: {len(qa_pairs)} Frage-Antwort-Paare")

# Datensatz-Statistiken
df_qa = pd.DataFrame(qa_pairs)
print(f"\n📈 Datensatz-Statistiken:")
print(f"  Kategorien: {df_qa['category'].value_counts().to_dict()}")
print(f"  Schwierigkeitsgrade: {df_qa['difficulty'].value_counts().to_dict()}")

# Erste Fragen anzeigen
print(f"\n🔍 Erste 3 Fragen:")
for i, qa in enumerate(qa_pairs[:3]):
    print(f"  {i+1}. {qa['question']}")
    print(f"     → {qa['expected_answer'][:100]}...")
    print(f"     Kategorie: {qa['category']}, Schwierigkeit: {qa['difficulty']}")
    print()


In [ ]:
# Batch-Evaluation durchführen
import time
from tqdm import tqdm

def run_evaluation(qa_pairs, pipeline, top_k=5):
    """Führt eine Batch-Evaluation durch."""
    if not pipeline.is_indexed:
        print("❌ Pipeline ist nicht indexiert")
        return None
    
    print(f"🔄 Starte Evaluation mit {len(qa_pairs)} Fragen...")
    
    results = []
    questions = [qa['question'] for qa in qa_pairs]
    
    # Batch-Query für bessere Performance
    start_time = time.time()
    batch_results = pipeline.batch_query(questions, top_k=top_k, show_progress=True)
    total_time = time.time() - start_time
    
    # Ergebnisse kombinieren
    for i, (qa, result) in enumerate(zip(qa_pairs, batch_results)):
        eval_result = {
            'id': qa['id'],
            'question': qa['question'],
            'expected_answer': qa['expected_answer'],
            'generated_answer': result['answer'],
            'category': qa['category'],
            'difficulty': qa['difficulty'],
            'retrieval_time': result['retrieval_time'],
            'generation_time': result['generation_time'],
            'total_time': result['total_time'],
            'context_count': len(result['context'])
        }
        results.append(eval_result)
    
    print(f"✅ Evaluation abgeschlossen in {total_time:.2f}s")
    print(f"⚡ Durchschnittliche Zeit pro Frage: {total_time/len(qa_pairs):.2f}s")
    
    return results

# Evaluation ausführen
if 'pipeline' in locals() and pipeline.is_indexed:
    print("🚀 Führe Evaluation durch...")
    evaluation_results = run_evaluation(qa_pairs, pipeline)
    
    if evaluation_results:
        # Ergebnisse als DataFrame
        df_results = pd.DataFrame(evaluation_results)
        
        print(f"\n📊 Evaluation-Ergebnisse:")
        print(f"  Fragen bearbeitet: {len(df_results)}")
        print(f"  Durchschnittliche Retrieval-Zeit: {df_results['retrieval_time'].mean():.2f}s")
        print(f"  Durchschnittliche Generation-Zeit: {df_results['generation_time'].mean():.2f}s")
        print(f"  Durchschnittliche Gesamt-Zeit: {df_results['total_time'].mean():.2f}s")
        
        # Nach Kategorie gruppieren
        category_stats = df_results.groupby('category').agg({
            'total_time': 'mean',
            'context_count': 'mean'
        }).round(2)
        
        print(f"\n📈 Statistiken nach Kategorie:")
        print(category_stats)
        
        # Erste Ergebnisse anzeigen
        print(f"\n🔍 Erste 3 Ergebnisse:")
        for i, result in enumerate(evaluation_results[:3]):
            print(f"\n  {i+1}. {result['question']}")
            print(f"     Erwartet: {result['expected_answer'][:100]}...")
            print(f"     Generiert: {result['generated_answer'][:100]}...")
            print(f"     Zeit: {result['total_time']:.2f}s")
else:
    print("❌ Pipeline nicht verfügbar oder nicht indexiert")
    print("💡 Führen Sie zuerst die Pipeline-Erstellung und Indexierung aus")


In [ ]:
# 8. Analyse

## Komponenten-Vergleich und Statistiken

Hier analysieren wir die Performance verschiedener Komponenten und erstellen Visualisierungen.


In [ ]:
# Visualisierungen erstellen
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting-Style setzen
plt.style.use('default')
sns.set_palette("husl")

def create_performance_plots(df_results):
    """Erstellt Performance-Visualisierungen."""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('RAG-Pipeline Performance Analyse', fontsize=16)
    
    # 1. Zeit-Verteilung
    axes[0, 0].hist(df_results['total_time'], bins=10, alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('Verteilung der Antwortzeiten')
    axes[0, 0].set_xlabel('Zeit (Sekunden)')
    axes[0, 0].set_ylabel('Anzahl Fragen')
    axes[0, 0].axvline(df_results['total_time'].mean(), color='red', linestyle='--', label=f'Mittelwert: {df_results["total_time"].mean():.2f}s')
    axes[0, 0].legend()
    
    # 2. Zeit nach Kategorie
    category_times = df_results.groupby('category')['total_time'].mean()
    axes[0, 1].bar(category_times.index, category_times.values, alpha=0.7)
    axes[0, 1].set_title('Durchschnittliche Antwortzeit nach Kategorie')
    axes[0, 1].set_xlabel('Kategorie')
    axes[0, 1].set_ylabel('Zeit (Sekunden)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. Retrieval vs Generation Zeit
    axes[1, 0].scatter(df_results['retrieval_time'], df_results['generation_time'], alpha=0.7)
    axes[1, 0].set_title('Retrieval-Zeit vs Generation-Zeit')
    axes[1, 0].set_xlabel('Retrieval-Zeit (Sekunden)')
    axes[1, 0].set_ylabel('Generation-Zeit (Sekunden)')
    
    # Trendlinie hinzufügen
    z = np.polyfit(df_results['retrieval_time'], df_results['generation_time'], 1)
    p = np.poly1d(z)
    axes[1, 0].plot(df_results['retrieval_time'], p(df_results['retrieval_time']), "r--", alpha=0.8)
    
    # 4. Kontext-Anzahl nach Schwierigkeit
    difficulty_context = df_results.groupby('difficulty')['context_count'].mean()
    axes[1, 1].bar(difficulty_context.index, difficulty_context.values, alpha=0.7)
    axes[1, 1].set_title('Durchschnittliche Kontext-Anzahl nach Schwierigkeit')
    axes[1, 1].set_xlabel('Schwierigkeit')
    axes[1, 1].set_ylabel('Anzahl Kontext-Dokumente')
    
    plt.tight_layout()
    plt.show()

# Performance-Analyse
if 'evaluation_results' in locals() and evaluation_results:
    print("📊 Erstelle Performance-Visualisierungen...")
    
    import numpy as np
    df_results = pd.DataFrame(evaluation_results)
    
    # Grundlegende Statistiken
    print(f"\n📈 Performance-Statistiken:")
    print(f"  Gesamtzeit - Mittelwert: {df_results['total_time'].mean():.2f}s ± {df_results['total_time'].std():.2f}s")
    print(f"  Retrieval-Zeit - Mittelwert: {df_results['retrieval_time'].mean():.2f}s ± {df_results['retrieval_time'].std():.2f}s")
    print(f"  Generation-Zeit - Mittelwert: {df_results['generation_time'].mean():.2f}s ± {df_results['generation_time'].std():.2f}s")
    print(f"  Kontext-Dokumente - Mittelwert: {df_results['context_count'].mean():.1f} ± {df_results['context_count'].std():.1f}")
    
    # Visualisierungen erstellen
    create_performance_plots(df_results)
    
    # Korrelationsanalyse
    print(f"\n🔗 Korrelationsanalyse:")
    correlation_matrix = df_results[['retrieval_time', 'generation_time', 'total_time', 'context_count']].corr()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.2f')
    plt.title('Korrelation zwischen Performance-Metriken')
    plt.tight_layout()
    plt.show()
    
    print("Interpretation:")
    print("- Hohe Korrelation zwischen Retrieval- und Gesamtzeit deutet auf Retrieval-Bottleneck hin")
    print("- Niedrige Korrelation zwischen Kontext-Anzahl und Zeit deutet auf effiziente Verarbeitung hin")
    
else:
    print("❌ Keine Evaluation-Ergebnisse verfügbar")
    print("💡 Führen Sie zuerst die Evaluation aus")


In [ ]:
# Komponenten-Debugging und Analyse
def analyze_pipeline_components():
    """Analysiert die Pipeline-Komponenten im Detail."""
    if 'pipeline' not in locals():
        print("❌ Pipeline nicht verfügbar")
        return
    
    print("🔍 Pipeline-Komponenten-Analyse:")
    
    # 1. Chunker-Analyse
    print(f"\n📝 Chunker-Analyse:")
    chunker_info = pipeline.get_component_info()['chunker']
    print(f"  Typ: {chunker_info}")
    
    # Beispiel-Chunking
    if os.path.exists(dsgvo_file):
        with open(dsgvo_file, 'r', encoding='utf-8') as f:
            sample_text = f.read()[:1000]  # Erste 1000 Zeichen
        
        chunks = pipeline.chunker.chunk_text(sample_text)
        print(f"  Beispiel-Chunks aus ersten 1000 Zeichen: {len(chunks)}")
        print(f"  Durchschnittliche Chunk-Länge: {sum(len(c) for c in chunks) / len(chunks):.0f} Zeichen")
        
        if chunks:
            print(f"  Erster Chunk: {chunks[0][:100]}...")
    
    # 2. Embedding-Analyse
    print(f"\n🔢 Embedding-Analyse:")
    embedding_info = pipeline.get_component_info()['embedding']
    print(f"  Typ: {embedding_info}")
    
    # Test-Embedding
    test_embedding = pipeline.embedding.embed_query("Test-Frage")
    print(f"  Embedding-Dimension: {len(test_embedding)}")
    print(f"  Embedding-Typ: {type(test_embedding)}")
    print(f"  Embedding-Range: [{min(test_embedding):.3f}, {max(test_embedding):.3f}]")
    
    # 3. Vector Store-Analyse
    print(f"\n🗄️  Vector Store-Analyse:")
    vector_store_info = pipeline.get_component_info()['vector_store']
    print(f"  Typ: {vector_store_info}")
    
    if hasattr(pipeline.vector_store, 'get_stats'):
        stats = pipeline.vector_store.get_stats()
        print(f"  Gespeicherte Dokumente: {stats.get('document_count', 'N/A')}")
        print(f"  Speichernutzung: {stats.get('memory_usage', 'N/A')}")
    
    # 4. Language Model-Analyse
    print(f"\n🤖 Language Model-Analyse:")
    llm_info = pipeline.get_component_info()['language_model']
    print(f"  Typ: {llm_info}")
    
    # Test-Generation
    test_context = ["Die DSGVO ist eine europäische Datenschutzverordnung."]
    test_response = pipeline.language_model.generate_with_context(
        "Was ist die DSGVO?", 
        test_context
    )
    print(f"  Test-Antwort: {test_response[:100]}...")

# Komponenten-Analyse ausführen
if 'pipeline' in locals():
    analyze_pipeline_components()
else:
    print("❌ Pipeline nicht verfügbar - bitte führen Sie zuerst die Pipeline-Erstellung aus")

# Speicher-Nutzung anzeigen
import psutil
import os

def show_memory_usage():
    """Zeigt die aktuelle Speicher-Nutzung an."""
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    
    print(f"\n💾 Speicher-Nutzung:")
    print(f"  RSS (Resident Set Size): {memory_info.rss / 1024 / 1024:.1f} MB")
    print(f"  VMS (Virtual Memory Size): {memory_info.vms / 1024 / 1024:.1f} MB")
    
    # System-Speicher
    system_memory = psutil.virtual_memory()
    print(f"  System-Speicher: {system_memory.used / 1024 / 1024 / 1024:.1f} GB / {system_memory.total / 1024 / 1024 / 1024:.1f} GB ({system_memory.percent}%)")

try:
    show_memory_usage()
except ImportError:
    print("💡 Installieren Sie 'psutil' für detaillierte Speicher-Analyse: pip install psutil")


In [ ]:
# 🎯 Zusammenfassung und nächste Schritte

## Was haben wir erreicht?

✅ **Erfolgreich implementiert:**
- Modulares RAG-System mit austauschbaren Komponenten
- Baseline-Konfiguration mit OpenAI-Komponenten
- Interaktive Jupyter-Umgebung für Experimente
- Systematische Evaluation mit QA-Datensatz
- Performance-Analyse und Visualisierungen

## Nächste Schritte für das Team

### 👥 Aufgabenverteilung (Phase 2-4)

**Person A - Chunking-Strategien:**
- Implementierung von `RecursiveChunker` und `SemanticChunker`
- Vergleich verschiedener Chunk-Größen und Overlap-Strategien
- Analyse der Auswirkungen auf Retrieval-Qualität

**Person B - Embedding-Methoden:**
- Integration von `SentenceTransformerEmbedding` und `HuggingFaceEmbedding`
- Vergleich verschiedener Embedding-Modelle
- Optimierung für deutsche DSGVO-Texte

**Person C - Vector Stores & Retrieval:**
- Implementierung von `ChromaVectorStore` und `FAISSVectorStore`
- Optimierung von Similarity-Metriken
- Skalierbarkeits-Tests

**Person D - Language Models & Generation:**
- Integration von `OllamaLanguageModel` und `HuggingFaceLanguageModel`
- Prompt-Engineering für bessere DSGVO-Antworten
- Evaluation der Antwortqualität

## 💡 Tipps für die Weiterarbeit

1. **Komponenten-Entwicklung:** Nutzen Sie die bestehenden Basisklassen als Vorlage
2. **Testing:** Verwenden Sie das Notebook für schnelle Prototyping-Zyklen
3. **Evaluation:** Erweitern Sie den QA-Datensatz für Ihre spezifischen Komponenten
4. **Dokumentation:** Aktualisieren Sie die README.md mit Ihren Erkenntnissen

## 🔧 Debugging-Tipps

- Verwenden Sie `analyze_pipeline_components()` für detaillierte Komponenten-Analyse
- Nutzen Sie `show_memory_usage()` zur Überwachung der Ressourcen
- Aktivieren Sie `return_context=True` bei Abfragen für besseres Debugging

## 📊 Experimentier-Framework

Das Notebook bietet alles was Sie brauchen:
- ✅ Konfiguration verschiedener Komponenten
- ✅ Batch-Evaluation für systematische Tests
- ✅ Visualisierungen für Performance-Analyse
- ✅ Interaktive Abfrage-Oberfläche

**Viel Erfolg bei Ihrer Forschung! 🚀**


In [ ]:
# ResearchRAG - Modulares RAG-System

## Projektübersicht

Dieses Notebook dient als Steuerungszentrale für das modulare RAG-System. Es ermöglicht:

- **Einfache Konfiguration** verschiedener Pipeline-Komponenten
- **Schnelle Experimente** mit unterschiedlichen Chunking-, Embedding-, Vector Store- und LLM-Strategien
- **Evaluierung** der Pipeline-Performance
- **Vergleich** verschiedener Konfigurationen

## Verwendung

1. **Konfiguration wählen** (Baseline oder Custom)
2. **Dokumente indexieren** (DSGVO-Text)
3. **Queries ausführen** und Ergebnisse analysieren
4. **Experimente durchführen** mit verschiedenen Komponenten

---

**Autoren:** FOM Research Team  
**Projekt:** Big Data Analyse - RAG-System Evaluierung  
**Datum:** Januar 2025


In [ ]:
# Installiere erforderliche Pakete (nur in Google Colab)
import sys
import subprocess

def install_packages():
    """Installiert alle erforderlichen Pakete für Google Colab."""
    packages = [
        "openai>=1.10.0",
        "sentence-transformers>=2.2.0",
        "chromadb>=0.4.0",
        "faiss-cpu>=1.7.4",
        "scikit-learn>=1.0.0",
        "numpy>=1.21.0",
        "pandas>=1.3.0",
        "python-dotenv>=0.19.0",
        "tqdm>=4.64.0"
    ]
    
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ {package} installiert")
        except subprocess.CalledProcessError:
            print(f"✗ Fehler bei Installation von {package}")

# Nur in Google Colab ausführen
if 'google.colab' in sys.modules:
    print("Google Colab erkannt - installiere Pakete...")
    install_packages()
    
    # Google Drive mounten für Persistierung
    from google.colab import drive
    drive.mount('/content/drive')
    
    print("✓ Setup für Google Colab abgeschlossen")
else:
    print("Lokale Umgebung erkannt - verwende requirements.txt")


In [ ]:
# Imports und Setup
import os
import sys
import json
import time
import warnings
from pathlib import Path
from typing import Dict, Any, List

# Warnings unterdrücken
warnings.filterwarnings('ignore')

# Lokale Imports
sys.path.append('.')
from src.config.pipeline_configs import PipelineConfig, get_baseline_config, get_alternative_configs
from src.core.rag_pipeline import RAGPipeline
from src.core.component_loader import ComponentLoader

# Umgebungsvariablen
from dotenv import load_dotenv
load_dotenv()

print("✓ Alle Module erfolgreich importiert")
print(f"✓ Python Version: {sys.version}")
print(f"✓ Arbeitsverzeichnis: {os.getcwd()}")

# API-Schlüssel prüfen
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  OPENAI_API_KEY nicht gesetzt!")
    print("   Setzen Sie Ihren API-Schlüssel:")
    print("   os.environ['OPENAI_API_KEY'] = 'your-api-key-here'")


In [ ]:
## 1. Pipeline-Konfiguration

Wählen Sie eine der verfügbaren Konfigurationen oder erstellen Sie eine benutzerdefinierte Konfiguration.


In [ ]:
# 1.1 Verfügbare Konfigurationen anzeigen
def show_available_configs():
    """Zeigt alle verfügbaren Pipeline-Konfigurationen."""
    configs = get_alternative_configs()
    
    print("🔧 Verfügbare Pipeline-Konfigurationen:\n")
    
    for name, config in configs.items():
        pipeline_info = config.get_pipeline_info()
        component_types = config.get_component_types()
        
        print(f"📋 {name.upper()}")
        print(f"   Beschreibung: {pipeline_info.get('description', 'Keine Beschreibung')}")
        print(f"   Chunker: {component_types['chunker']}")
        print(f"   Embedding: {component_types['embedding']}")
        print(f"   Vector Store: {component_types['vector_store']}")
        print(f"   Language Model: {component_types['language_model']}")
        print()

show_available_configs()


In [ ]:
# 1.2 Konfiguration wählen
# Ändern Sie hier die gewünschte Konfiguration
SELECTED_CONFIG = "baseline"  # Optionen: "baseline", "recursive_chunker", "sentence_transformer", "chroma_store", "gpt4_model"

# Konfiguration laden
configs = get_alternative_configs()
if SELECTED_CONFIG not in configs:
    print(f"❌ Konfiguration '{SELECTED_CONFIG}' nicht gefunden!")
    print(f"Verfügbare Optionen: {list(configs.keys())}")
    selected_config = get_baseline_config()
else:
    selected_config = configs[SELECTED_CONFIG]

print(f"✅ Gewählte Konfiguration: {SELECTED_CONFIG}")
print(f"📝 Beschreibung: {selected_config.get_pipeline_info().get('description', 'Keine Beschreibung')}")
print("\n🔧 Komponenten:")
for component, type_name in selected_config.get_component_types().items():
    print(f"   {component}: {type_name}")


In [ ]:
## 2. Pipeline initialisieren

Erstellen Sie die RAG-Pipeline mit der gewählten Konfiguration.


In [ ]:
# 2.1 Pipeline erstellen
try:
    # API-Schlüssel setzen falls noch nicht vorhanden
    if not os.getenv("OPENAI_API_KEY"):
        # Beispiel für API-Schlüssel setzen (ersetzen Sie durch Ihren echten Schlüssel)
        # os.environ['OPENAI_API_KEY'] = 'sk-your-api-key-here'
        print("⚠️  Bitte setzen Sie Ihren OpenAI API-Schlüssel!")
        print("   os.environ['OPENAI_API_KEY'] = 'sk-your-api-key-here'")
    
    # Pipeline erstellen
    print("🚀 Erstelle RAG-Pipeline...")
    pipeline = RAGPipeline(selected_config)
    
    print("✅ Pipeline erfolgreich erstellt!")
    print(f"📊 Pipeline-Info: {pipeline.get_pipeline_info()}")
    
except Exception as e:
    print(f"❌ Fehler beim Erstellen der Pipeline: {e}")
    print("💡 Tipp: Überprüfen Sie Ihren OpenAI API-Schlüssel")


In [ ]:
## 3. Dokumente indexieren

Laden und indexieren Sie die DSGVO-Dokumente.


In [ ]:
# 3.1 DSGVO-Dokument laden und indexieren
try:
    # Dokumentpfad
    dsgvo_path = "data/raw/dsgvo.txt"
    
    if not os.path.exists(dsgvo_path):
        print(f"❌ DSGVO-Datei nicht gefunden: {dsgvo_path}")
        print("💡 Stellen Sie sicher, dass die DSGVO-Datei im data/raw/ Verzeichnis liegt")
    else:
        # Dokument laden
        print("📖 Lade DSGVO-Dokument...")
        documents = pipeline.load_documents_from_file(dsgvo_path)
        print(f"✅ {len(documents)} Dokument(e) geladen")
        print(f"📏 Dokumentlänge: {len(documents[0]):,} Zeichen")
        
        # Indexierung
        print("\n🔄 Starte Indexierung...")
        start_time = time.time()
        
        indexing_stats = pipeline.index_documents(documents, show_progress=True)
        
        end_time = time.time()
        
        print(f"\n✅ Indexierung abgeschlossen!")
        print(f"⏱️  Gesamtzeit: {end_time - start_time:.2f} Sekunden")
        print(f"📊 Statistiken:")
        for key, value in indexing_stats.items():
            if isinstance(value, float):
                print(f"   {key}: {value:.2f}")
            else:
                print(f"   {key}: {value}")
        
except Exception as e:
    print(f"❌ Fehler beim Indexieren: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
## 4. RAG-System testen

Führen Sie Beispiel-Queries aus und analysieren Sie die Ergebnisse.


In [ ]:
# 4.1 Beispiel-Queries definieren
sample_questions = [
    "Was ist die maximale Geldbuße nach Art. 83 DSGVO?",
    "Welche Rechtsgrundlagen für die Verarbeitung gibt es?",
    "Wie läuft das Verfahren bei einer Datenschutz-Folgenabschätzung ab?",
    "Welche Rechte haben betroffene Personen?",
    "Was sind die Grundsätze der Datenverarbeitung?"
]

print("📝 Beispiel-Fragen:")
for i, question in enumerate(sample_questions, 1):
    print(f"{i}. {question}")

print(f"\n💡 Sie können auch eigene Fragen in der nächsten Zelle stellen!")


In [ ]:
# 4.2 Einzelne Query ausführen
def ask_question(question: str, show_context: bool = False):
    """Führt eine einzelne Query aus und zeigt das Ergebnis."""
    print(f"❓ Frage: {question}")
    print("=" * 80)
    
    try:
        # Query ausführen
        result = pipeline.query(question, return_context=show_context)
        
        print(f"💬 Antwort:")
        print(result['answer'])
        print()
        
        print(f"📊 Metadaten:")
        print(f"   ⏱️  Query-Zeit: {result['query_time']:.2f}s")
        print(f"   🔍 Gefundene Kontexte: {result['retrieval_count']}")
        print(f"   🔧 Pipeline: {result['pipeline_config']}")
        
        if show_context and 'retrieved_contexts' in result:
            print(f"\n📖 Gefundene Kontexte:")
            for i, context in enumerate(result['retrieved_contexts'], 1):
                print(f"\n   Kontext {i} (Score: {context['score']:.3f}):")
                print(f"   {context['text'][:200]}...")
        
    except Exception as e:
        print(f"❌ Fehler bei der Query: {e}")
    
    print("=" * 80)

# Beispiel-Query ausführen
ask_question(sample_questions[0], show_context=True)


In [ ]:
# 4.3 Eigene Frage stellen
# Ändern Sie hier Ihre eigene Frage
CUSTOM_QUESTION = "Welche Pflichten haben Auftragsverarbeiter?"

print("🎯 Ihre eigene Frage:")
ask_question(CUSTOM_QUESTION, show_context=False)


In [ ]:
# 4.4 Batch-Verarbeitung aller Beispiel-Fragen
print("🔄 Verarbeite alle Beispiel-Fragen...")
batch_results = pipeline.batch_query(sample_questions, show_progress=True)

print("\n📊 Zusammenfassung der Ergebnisse:")
print("=" * 80)

total_time = sum(result['query_time'] for result in batch_results)
avg_time = total_time / len(batch_results)

for i, (question, result) in enumerate(zip(sample_questions, batch_results), 1):
    print(f"\n{i}. {question}")
    print(f"   💬 Antwort: {result['answer'][:100]}...")
    print(f"   ⏱️  Zeit: {result['query_time']:.2f}s")
    print(f"   🔍 Kontexte: {result['retrieval_count']}")

print(f"\n📈 Gesamt-Statistiken:")
print(f"   🎯 Verarbeitete Fragen: {len(batch_results)}")
print(f"   ⏱️  Gesamtzeit: {total_time:.2f}s")
print(f"   📊 Durchschnittliche Zeit: {avg_time:.2f}s")
print(f"   🚀 Fragen pro Sekunde: {len(batch_results)/total_time:.2f}")


In [ ]:
## 5. Pipeline-Analyse und Debugging

Analysieren Sie die Pipeline-Komponenten und deren Verhalten.


In [ ]:
# 5.1 Detaillierte Pipeline-Informationen
print("🔍 Detaillierte Pipeline-Analyse:")
print("=" * 80)

# Pipeline-Info
pipeline_info = pipeline.get_pipeline_info()
print(f"📋 Pipeline-Informationen:")
for key, value in pipeline_info.items():
    print(f"   {key}: {value}")

print("\n" + "=" * 80)

# Komponenten-Info
component_info = pipeline.get_component_info()
print(f"🧩 Komponenten-Details:")

for component_name, info in component_info.items():
    print(f"\n📦 {component_name.upper()}:")
    print(f"   Typ: {info['type']}")
    print(f"   Konfiguration:")
    for key, value in info['config'].items():
        print(f"      {key}: {value}")
    
    if 'stats' in info and info['stats']:
        print(f"   Statistiken:")
        for key, value in info['stats'].items():
            print(f"      {key}: {value}")

print("\n" + "=" * 80)


In [ ]:
# 5.2 Chunking-Analyse
print("📄 Chunking-Analyse:")
print("=" * 50)

# Beispiel-Text chunken
sample_text = """
Art. 1 Gegenstand und Ziele

(1) Diese Verordnung enthält Vorschriften zum Schutz natürlicher Personen bei der Verarbeitung personenbezogener Daten und zum freien Verkehr solcher Daten.

(2) Diese Verordnung schützt die Grundrechte und Grundfreiheiten natürlicher Personen und insbesondere deren Recht auf Schutz personenbezogener Daten.

(3) Der freie Verkehr personenbezogener Daten in der Union darf aus Gründen des Schutzes natürlicher Personen bei der Verarbeitung personenbezogener Daten weder eingeschränkt noch untersagt werden.
"""

chunks = pipeline.chunker.chunk_text(sample_text.strip())

print(f"📝 Original-Text ({len(sample_text)} Zeichen):")
print(sample_text[:200] + "...")

print(f"\n🔪 Chunks ({len(chunks)} Stück):")
for i, chunk in enumerate(chunks, 1):
    print(f"\nChunk {i} ({len(chunk)} Zeichen):")
    print(f"'{chunk[:100]}...'")

print(f"\n📊 Chunking-Statistiken:")
print(f"   Anzahl Chunks: {len(chunks)}")
print(f"   Durchschnittliche Chunk-Länge: {sum(len(c) for c in chunks) / len(chunks):.1f} Zeichen")
print(f"   Kürzester Chunk: {min(len(c) for c in chunks)} Zeichen")
print(f"   Längster Chunk: {max(len(c) for c in chunks)} Zeichen")


In [ ]:
## 6. Experimentieren mit verschiedenen Konfigurationen

Testen Sie verschiedene Pipeline-Konfigurationen und vergleichen Sie die Ergebnisse.


In [ ]:
# 6.1 Vergleich verschiedener Konfigurationen
def compare_configurations(test_question: str = "Was ist die maximale Geldbuße nach Art. 83 DSGVO?"):
    """Vergleicht verschiedene Pipeline-Konfigurationen."""
    
    configs_to_test = ["baseline"]  # Nur Baseline für jetzt, da andere Komponenten noch nicht implementiert
    
    print(f"🔬 Experiment: Vergleich verschiedener Konfigurationen")
    print(f"❓ Test-Frage: {test_question}")
    print("=" * 80)
    
    results = {}
    
    for config_name in configs_to_test:
        print(f"\n🧪 Teste Konfiguration: {config_name}")
        
        try:
            # Konfiguration laden
            configs = get_alternative_configs()
            config = configs[config_name]
            
            # Neue Pipeline erstellen
            test_pipeline = RAGPipeline(config)
            
            # Dokument indexieren (falls noch nicht geschehen)
            if not test_pipeline.is_indexed:
                documents = test_pipeline.load_documents_from_file("data/raw/dsgvo.txt")
                test_pipeline.index_documents(documents, show_progress=False)
            
            # Query ausführen
            start_time = time.time()
            result = test_pipeline.query(test_question)
            end_time = time.time()
            
            results[config_name] = {
                'answer': result['answer'],
                'query_time': result['query_time'],
                'retrieval_count': result['retrieval_count'],
                'components': config.get_component_types()
            }
            
            print(f"   ✅ Erfolgreich getestet")
            print(f"   ⏱️  Zeit: {result['query_time']:.2f}s")
            print(f"   💬 Antwort: {result['answer'][:100]}...")
            
        except Exception as e:
            print(f"   ❌ Fehler: {e}")
            results[config_name] = {'error': str(e)}
    
    print("\n" + "=" * 80)
    print("📊 Vergleichsergebnisse:")
    
    for config_name, result in results.items():
        if 'error' not in result:
            print(f"\n🔧 {config_name.upper()}:")
            print(f"   ⏱️  Query-Zeit: {result['query_time']:.2f}s")
            print(f"   🔍 Retrieval-Count: {result['retrieval_count']}")
            print(f"   🧩 Komponenten: {result['components']}")
            print(f"   💬 Antwort: {result['answer'][:150]}...")
    
    return results

# Experiment ausführen
experiment_results = compare_configurations()


In [ ]:
## 7. Nächste Schritte

Dieses Notebook bietet eine solide Basis für Ihre RAG-System-Experimente. Hier sind die nächsten Schritte für Ihre Forschung:

### Für Person A (Chunking-Strategien):
- Implementieren Sie `RecursiveChunker` in `src/components/chunkers/recursive_chunker.py`
- Experimentieren Sie mit verschiedenen Chunk-Größen und Überlappungen
- Testen Sie semantisches Chunking

### Für Person B (Embedding-Verfahren):
- Implementieren Sie `SentenceTransformerEmbedding` in `src/components/embeddings/sentence_transformer_embedding.py`
- Vergleichen Sie verschiedene Embedding-Modelle
- Analysieren Sie die Embedding-Qualität

### Für Person C (Vector Stores & Retrieval):
- Implementieren Sie `ChromaVectorStore` in `src/components/vector_stores/chroma_vector_store.py`
- Testen Sie verschiedene Ähnlichkeitsmetriken
- Optimieren Sie Retrieval-Parameter

### Für Person D (Language Models & Generation):
- Implementieren Sie lokale LLM-Integration (Ollama)
- Experimentieren Sie mit verschiedenen Prompt-Strategien
- Optimieren Sie Generation-Parameter

### Gemeinsame Aufgaben:
- Erstellen Sie QA-Datensätze für systematische Evaluierung
- Implementieren Sie Evaluierungsmetriken
- Dokumentieren Sie Ihre Experimente
